In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

In [2]:
image_dir = Path(r"C:\Users\Fábio\Desktop\Banco de Imagens Completo\original")

In [3]:
# Get filepaths and labels (imagens .jpg)
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# DataFrame
image_df = pd.concat([filepaths, labels], axis=1)

# Drop GT images (se houver)
image_df['Label'] = image_df['Label'].apply(lambda x: np.NaN if x[-2:] == 'GT' else x)
image_df = image_df.dropna(axis=0).reset_index(drop=True)

# Amostragem: até 500 por classe (se tiver menos, amostra com replace=True p/ completar)
samples = []
for category in image_df['Label'].unique():
    category_slice = image_df.query("Label == @category")
    samples.append(
        category_slice.sample(500, random_state=1, replace=(len(category_slice) < 500))
    )

image_df = (
    pd.concat(samples, axis=0)
      .sample(frac=1.0, random_state=1)
      .reset_index(drop=True)
)


In [4]:
image_df.head()

,Filepath,Label
0,C:\Users\Fábio\Desktop\Banco de Imagens Comple...,sample_data
1,C:\Users\Fábio\Desktop\Banco de Imagens Comple...,Poecilia_vivipara
2,C:\Users\Fábio\Desktop\Banco de Imagens Comple...,Hoplias_Malabaricus
3,C:\Users\Fábio\Desktop\Banco de Imagens Comple...,sample_data
4,C:\Users\Fábio\Desktop\Banco de Imagens Comple...,Imparfinis_schubarti


In [ ]:
# Limpar espaços
image_df['Label'] = image_df['Label'].str.strip()

# Lista oficial de 24 classes
ALLOWED_CLASSES = [
    "Ancistrus_albino",
    "Astyanax_altiparanae",
    "Bryconamericus_iheringii",
    "Characidium_zebra",
    "Cichlasoma_paranaense",
    "Crenicichla_britskii",
    "Crenicichla_haroldoi",
    "Geophagus_brasiliensis",
    "Gymnotus_inaequilabiatus",
    "Hoplias_Malabaricus",
    "Hoplosternum_littorale",
    "Hypostomus_ancistroides",
    "Imparfinis_schubarti",
    "Leporinus_striatus",
    "Olygosarcus_paranensis",
    "Phalloceros_caudimaculatus",
    "Poecilia_reticulada",
    "Poecilia_vivipara",
    "Psalidodon_bockmanni",
    "Rhamdia_quelen",
    "Serrapinnus_notomelas",
    "Serrasalmus_maculatus",
    "Synbranchus_marmoratus",
    "Trichomyterus_sp",
]

# Filtrar as imagens a partir da lista oficial de classes (24 classes)
image_df = image_df[image_df['Label'].isin(ALLOWED_CLASSES)].reset_index(drop=True)

print("Classes únicas após filtro:", image_df['Label'].nunique())
print(sorted(image_df['Label'].unique()))

# Split estratificado (70/30)
train_df, test_df = train_test_split(
    image_df, train_size=0.7, shuffle=True, random_state=1, stratify=image_df['Label']
)

# Ordem fixa das classes para os geradores
class_names = ALLOWED_CLASSES[:]  # mantém a ordem definida acima
num_classes = len(class_names)
print("num_classes =", num_classes)


Classes únicas após filtro: 24
['Ancistrus_albino', 'Astyanax_altiparanae', 'Bryconamericus_iheringii', 'Characidium_zebra', 'Cichlasoma_paranaense', 'Crenicichla_britskii', 'Crenicichla_haroldoi', 'Geophagus_brasiliensis', 'Gymnotus_inaequilabiatus', 'Hoplias_Malabaricus', 'Hoplosternum_littorale', 'Hypostomus_ancistroides', 'Imparfinis_schubarti', 'Leporinus_striatus', 'Olygosarcus_paranensis', 'Phalloceros_caudimaculatus', 'Poecilia_reticulada', 'Poecilia_vivipara', 'Psalidodon_bockmanni', 'Rhamdia_quelen', 'Serrapinnus_notomelas', 'Serrasalmus_maculatus', 'Synbranchus_marmoratus', 'Trichomyterus_sp']
num_classes = 24


In [6]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2  # 20% do train para validação
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    classes=class_names,               # <- fixa as 24 classes
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    classes=class_names,               # <- idem
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    classes=class_names,               # <- idem
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

print("Treino:", len(train_images.class_indices), train_images.class_indices)
print("Val   :", len(val_images.class_indices),   val_images.class_indices)
print("Teste :", len(test_images.class_indices),  test_images.class_indices)


Found 6720 validated image filenames belonging to 24 classes.
Found 1680 validated image filenames belonging to 24 classes.
Found 3600 validated image filenames belonging to 24 classes.
Treino: 24 {'Ancistrus_albino': 0, 'Astyanax_altiparanae': 1, 'Bryconamericus_iheringii': 2, 'Characidium_zebra': 3, 'Cichlasoma_paranaense': 4, 'Crenicichla_britskii': 5, 'Crenicichla_haroldoi': 6, 'Geophagus_brasiliensis': 7, 'Gymnotus_inaequilabiatus': 8, 'Hoplias_Malabaricus': 9, 'Hoplosternum_littorale': 10, 'Hypostomus_ancistroides': 11, 'Imparfinis_schubarti': 12, 'Leporinus_striatus': 13, 'Olygosarcus_paranensis': 14, 'Phalloceros_caudimaculatus': 15, 'Poecilia_reticulada': 16, 'Poecilia_vivipara': 17, 'Psalidodon_bockmanni': 18, 'Rhamdia_quelen': 19, 'Serrapinnus_notomelas': 20, 'Serrasalmus_maculatus': 21, 'Synbranchus_marmoratus': 22, 'Trichomyterus_sp': 23}
Val   : 24 {'Ancistrus_albino': 0, 'Astyanax_altiparanae': 1, 'Bryconamericus_iheringii': 2, 'Characidium_zebra': 3, 'Cichlasoma_paranae

In [7]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
pretrained_model.trainable = False

inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)  # 24 saídas

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()  # confira última camada: (None, 24)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,441,560 (9.31 MB)

 Trainable params: 183,576 (717.09 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [8]:
from pathlib import Path
import json, os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score, cohen_kappa_score, matthews_corrcoef

REPORTS_DIR = Path("reports")
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# Salva histórico (loss/acc) por época
csv_logger = tf.keras.callbacks.CSVLogger(str(REPORTS_DIR/"history.csv"), append=False)

class ValMetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_data, class_names, outdir=REPORTS_DIR, every_n_epochs=1):
        super().__init__()
        self.val_data = val_data
        self.class_names = class_names
        self.outdir = Path(outdir)
        self.every_n_epochs = every_n_epochs

    def _save_confusion_matrix_png(self, cm, epoch):
        fig, ax = plt.subplots(figsize=(10, 9))
        im = ax.imshow(cm, interpolation='nearest')
        ax.set_title(f'Matriz de Confusão (Val) — Época {epoch+1}')
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        tick_marks = np.arange(len(self.class_names))
        ax.set_xticks(tick_marks); ax.set_xticklabels(self.class_names, rotation=90, fontsize=8)
        ax.set_yticks(tick_marks); ax.set_yticklabels(self.class_names, fontsize=8)
        ax.set_xlabel('Previsto'); ax.set_ylabel('Real')
        # anotação por célula (opcional, mas útil)
        thresh = cm.max() / 2.0
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j, i, int(cm[i, j]),
                        ha="center", va="center",
                        color="white" if cm[i, j] > thresh else "black", fontsize=7)
        plt.tight_layout()
        fig.savefig(self.outdir / f"val_confusion_matrix_epoch_{epoch+1}.png", dpi=180, bbox_inches='tight')
        plt.close(fig)

    def _per_class_table(self, cm):
        tp = np.diag(cm).astype(float)
        fp = cm.sum(axis=0) - tp
        fn = cm.sum(axis=1) - tp
        tn = cm.sum() - (tp + fp + fn)
        precision = np.divide(tp, (tp + fp), out=np.zeros_like(tp), where=(tp+fp)!=0)
        recall    = np.divide(tp, (tp + fn), out=np.zeros_like(tp), where=(tp+fn)!=0)
        f1        = np.divide(2*precision*recall, (precision+recall), out=np.zeros_like(tp), where=(precision+recall)!=0)
        specificity = np.divide(tn, (tn+fp), out=np.zeros_like(tp), where=(tn+fp)!=0)
        support = cm.sum(axis=1)
        import pandas as pd
        return pd.DataFrame({
            "class": self.class_names,
            "support": support.astype(int),
            "TP": tp.astype(int), "FP": fp.astype(int), "FN": fn.astype(int), "TN": tn.astype(int),
            "precision": precision, "recall": recall, "f1": f1, "specificity": specificity
        })

    def on_epoch_end(self, epoch, logs=None):
        if (epoch % self.every_n_epochs) != 0:
            return

        y_true = self.val_data.classes
        y_prob = self.model.predict(self.val_data, verbose=0)
        y_pred = np.argmax(y_prob, axis=1)

        cm = confusion_matrix(y_true, y_pred, labels=range(len(self.class_names)))
        bal_acc = balanced_accuracy_score(y_true, y_pred)
        kappa = cohen_kappa_score(y_true, y_pred)
        mcc = matthews_corrcoef(y_true, y_pred)

        # salva matriz de confusão (png) e tabela por classe (csv)
        self._save_confusion_matrix_png(cm, epoch)
        per_class = self._per_class_table(cm)
        per_class.to_csv(self.outdir / f"val_per_class_epoch_{epoch+1}.csv", index=False, encoding="utf-8")

        # classification report (txt) + métricas globais (json)
        report_txt = classification_report(
            y_true, y_pred, target_names=self.class_names, digits=4
        )
        (self.outdir / f"val_classification_report_epoch_{epoch+1}.txt").write_text(report_txt, encoding="utf-8")

        json.dump({
            "epoch": int(epoch+1),
            "balanced_accuracy": float(bal_acc),
            "kappa": float(kappa),
            "mcc": float(mcc)
        }, open(self.outdir / f"val_metrics_epoch_{epoch+1}.json", "w"), indent=2, ensure_ascii=False)

        print(f"\n[ValMetrics] Época {epoch+1} — Balanced Acc={bal_acc:.4f} | Kappa={kappa:.4f} | MCC={mcc:.4f}\n")


In [9]:
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        csv_logger,
        ValMetricsCallback(val_images, class_names, REPORTS_DIR, every_n_epochs=1),
    ]
)


c:\Users\Fábio\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 688ms/step - accuracy: 0.6961 - loss: 1.1927
[ValMetrics] Época 1 — Balanced Acc=0.9698 | Kappa=0.9677 | MCC=0.9678

210/210 ━━━━━━━━━━━━━━━━━━━━ 232s 1s/step - accuracy: 0.8737 - loss: 0.5081 - val_accuracy: 0.9690 - val_loss: 0.1071
Epoch 2/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 654ms/step - accuracy: 0.9856 - loss: 0.0574
[ValMetrics] Época 2 — Balanced Acc=0.9847 | Kappa=0.9845 | MCC=0.9845

210/210 ━━━━━━━━━━━━━━━━━━━━ 212s 1s/step - accuracy: 0.9874 - loss: 0.0475 - val_accuracy: 0.9851 - val_loss: 0.0566
Epoch 3/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 648ms/step - accuracy: 0.9973 - loss: 0.0160
[ValMetrics] Época 3 — Balanced Acc=0.9544 | Kappa=0.9484 | MCC=0.9492

210/210 ━━━━━━━━━━━━━━━━━━━━ 210s 1s/step - accuracy: 0.9939 - loss: 0.0227 - val_accuracy: 0.9506 - val_loss: 0.1399
Epoch 4/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 648ms/step - accuracy: 0.9926 - loss: 0.0316
[ValMetrics] Época 4 — Balanced Acc=0.9880 | Kappa=0.9863 | MCC=0.9864

210/21

In [10]:
test_loss, test_acc = model.evaluate(test_images)
print("Test loss:", test_loss)
print("Test acc :", test_acc)


c:\Users\Fábio\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


113/113 ━━━━━━━━━━━━━━━━━━━━ 78s 689ms/step - accuracy: 0.9964 - loss: 0.0127
Test loss: 0.012749459594488144
Test acc : 0.9963889122009277


In [11]:
len(set(train_df['Filepath']) & set(test_df['Filepath']))


0

In [12]:
from sklearn.metrics import precision_recall_fscore_support

# Probabilidades e predições no teste
y_true = test_images.classes
y_prob = model.predict(test_images, verbose=0)
y_pred = np.argmax(y_prob, axis=1)

# Matriz de confusão (teste)
cm_test = confusion_matrix(y_true, y_pred, labels=range(num_classes))

# Salvar imagem da matriz de confusão do TESTE
def save_cm_png(cm, class_names, path_png, title="Matriz de Confusão (Teste)"):
    fig, ax = plt.subplots(figsize=(10, 9))
    im = ax.imshow(cm, interpolation='nearest')
    ax.set_title(title)
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ticks = np.arange(len(class_names))
    ax.set_xticks(ticks); ax.set_xticklabels(class_names, rotation=90, fontsize=8)
    ax.set_yticks(ticks); ax.set_yticklabels(class_names, fontsize=8)
    ax.set_xlabel('Previsto'); ax.set_ylabel('Real')
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, int(cm[i, j]),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black", fontsize=7)
    plt.tight_layout()
    fig.savefig(path_png, dpi=180, bbox_inches='tight')
    plt.close(fig)

save_cm_png(cm_test, class_names, REPORTS_DIR/"test_confusion_matrix.png")

# Classification report (teste)
report_test = classification_report(y_true, y_pred, target_names=class_names, digits=4)
(Path(REPORTS_DIR/"test_classification_report.txt")).write_text(report_test, encoding="utf-8")

# Métricas globais adicionais (teste)
bal_acc = balanced_accuracy_score(y_true, y_pred)
kappa = cohen_kappa_score(y_true, y_pred)
mcc = matthews_corrcoef(y_true, y_pred)

json.dump({
    "test_loss": float(test_loss),
    "test_acc": float(test_acc),
    "balanced_accuracy": float(bal_acc),
    "kappa": float(kappa),
    "mcc": float(mcc)
}, open(REPORTS_DIR/"test_metrics_summary.json", "w"), indent=2, ensure_ascii=False)

print("=== TESTE ===")
print(f"Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")
print(f"Balanced Acc: {bal_acc:.4f} | Kappa: {kappa:.4f} | MCC: {mcc:.4f}")
print("Classification report salvo em reports/test_classification_report.txt")
print("Matriz de confusão salva em reports/test_confusion_matrix.png")


=== TESTE ===
Test loss: 0.0127 | Test acc: 0.9964
Balanced Acc: 0.9964 | Kappa: 0.9962 | MCC: 0.9962
Classification report salvo em reports/test_classification_report.txt
Matriz de confusão salva em reports/test_confusion_matrix.png


In [13]:
import pandas as pd
# Mapas úteis
idx_to_class = {v:k for k,v in test_images.class_indices.items()}
filepaths_test = np.array(test_images.filepaths)
true_labels_str = np.array([idx_to_class[i] for i in y_true])
pred_labels_str = np.array([idx_to_class[i] for i in y_pred])

# prob da classe prevista e da classe real
prob_pred_class = y_prob[np.arange(len(y_prob)), y_pred]
prob_true_class = y_prob[np.arange(len(y_prob)), y_true]

df_test_preds = pd.DataFrame({
    "filepath": filepaths_test,
    "true_idx": y_true,
    "true_label": true_labels_str,
    "pred_idx": y_pred,
    "pred_label": pred_labels_str,
    "prob_pred_class": prob_pred_class,
    "prob_true_class": prob_true_class
})

# Falsos Positivos (FP): previsto = C, real != C
fps = []
fns = []
for c_idx, c_name in enumerate(class_names):
    # FP da classe c: modelo disse "c", mas o real era outro
    df_fp_c = df_test_preds[(df_test_preds["pred_idx"] == c_idx) & (df_test_preds["true_idx"] != c_idx)].copy()
    df_fp_c["as_positive_class"] = c_name
    fps.append(df_fp_c)

    # FN da classe c: real era "c", mas modelo previu outra
    df_fn_c = df_test_preds[(df_test_preds["true_idx"] == c_idx) & (df_test_preds["pred_idx"] != c_idx)].copy()
    df_fn_c["missed_positive_class"] = c_name
    fns.append(df_fn_c)

df_fp_all = pd.concat(fps, axis=0).sort_values(["as_positive_class","prob_pred_class"], ascending=[True, False]).reset_index(drop=True)
df_fn_all = pd.concat(fns, axis=0).sort_values(["missed_positive_class","prob_true_class"], ascending=[True, False]).reset_index(drop=True)

df_fp_all.to_csv(REPORTS_DIR/"test_false_positives.csv", index=False, encoding="utf-8")
df_fn_all.to_csv(REPORTS_DIR/"test_false_negatives.csv", index=False, encoding="utf-8")

print("FP e FN salvos em:")
print(" - reports/test_false_positives.csv")
print(" - reports/test_false_negatives.csv")


FP e FN salvos em:
 - reports/test_false_positives.csv
 - reports/test_false_negatives.csv


In [14]:
# Matriz de confusão em CSV
pd.DataFrame(cm_test, index=class_names, columns=class_names).to_csv(REPORTS_DIR/"test_confusion_matrix.csv", encoding="utf-8")

# Tabela por classe (teste)
def per_class_table(cm, class_names):
    tp = np.diag(cm).astype(float)
    fp = cm.sum(axis=0) - tp
    fn = cm.sum(axis=1) - tp
    tn = cm.sum() - (tp + fp + fn)
    precision = np.divide(tp, (tp + fp), out=np.zeros_like(tp), where=(tp+fp)!=0)
    recall    = np.divide(tp, (tp + fn), out=np.zeros_like(tp), where=(tp+fn)!=0)
    f1        = np.divide(2*precision*recall, (precision+recall), out=np.zeros_like(tp), where=(precision+recall)!=0)
    specificity = np.divide(tn, (tn+fp), out=np.zeros_like(tp), where=(tn+fp)!=0)
    support = cm.sum(axis=1)
    return pd.DataFrame({
        "class": class_names,
        "support": support.astype(int),
        "TP": tp.astype(int), "FP": fp.astype(int), "FN": fn.astype(int), "TN": tn.astype(int),
        "precision": precision, "recall": recall, "f1": f1, "specificity": specificity
    })

per_class_test = per_class_table(cm_test, class_names)
per_class_test.to_csv(REPORTS_DIR/"test_per_class.csv", index=False, encoding="utf-8")


In [15]:
# depois do fit:
model.save("model_mobilenetv2_paranapanema.h5")

# salvar o mapeamento de classes do gerador de treino
import json
with open("class_indices.json", "w", encoding="utf-8") as f:
    json.dump({k:int(v) for k,v in train_images.class_indices.items()}, f, ensure_ascii=False, indent=2)


In [23]:
from pathlib import Path
import json
import numpy as np
import tensorflow as tf

# carregar modelo e classes
model = tf.keras.models.load_model("model_mobilenetv2_paranapanema.h5")
with open("class_indices.json", "r", encoding="utf-8") as f:
    class_indices = json.load(f)
idx_to_class = {v: k for k, v in class_indices.items()}

IMG_SIZE = (224, 224)

def load_and_preprocess(img_path: str):
    img_bytes = tf.io.read_file(img_path)
    img = tf.image.decode_image(img_bytes, channels=3, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE, method=tf.image.ResizeMethod.BICUBIC)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img * 255.0)
    return tf.expand_dims(img, 0)  # batch de 1

def predict_image(img_path: str, top_k: int = 1):
    x = load_and_preprocess(img_path)
    probs = model.predict(x, verbose=0)[0]
    top_idx = np.argsort(-probs)[:top_k]
    return [(idx_to_class[i], float(probs[i])) for i in top_idx]

# lista de imagens para testar
imgs = [
    r"C:\Users\Fábio\Desktop\Banco de Imagens Completo\Imagens para teste no app\Astyanax_altiparanae\teste01.jpg",
    r"C:\Users\Fábio\Desktop\Banco de Imagens Completo\Imagens para teste no app\Astyanax_altiparanae\teste02.jpg",
    r"C:\Users\Fábio\Desktop\Banco de Imagens Completo\Imagens para teste no app\Astyanax_altiparanae\teste03.jpg",
    r"C:\Users\Fábio\Desktop\Banco de Imagens Completo\Imagens para teste no app\Astyanax_altiparanae\teste04.jpg",
    r"C:\Users\Fábio\Desktop\Banco de Imagens Completo\Imagens para teste no app\Astyanax_altiparanae\teste05.jpg",

]

# predizer todas e imprimir
for img_path in imgs:
    result = predict_image(img_path, top_k=1)
    print(f"{Path(img_path).name} -> {result}")


teste01.jpg -> [('Astyanax_altiparanae', 0.9999663829803467)]
teste02.jpg -> [('Astyanax_altiparanae', 0.9999696016311646)]
teste03.jpg -> [('Astyanax_altiparanae', 0.9997878670692444)]
teste04.jpg -> [('Astyanax_altiparanae', 0.9999728202819824)]
teste05.jpg -> [('Astyanax_altiparanae', 0.9999953508377075)]
